# Limpieza datos 
**Objetivo:** limpiar y preparar los datos del mes de diciembre 2019. 
Se conservarán las columnas: 
`tpep_pickup_datetime`, `tpep_dropoff_datetime`, `passenger_count`,
`trip_distance`, `pulocationid`, `dolocationid`, `payment_type`,
`fare_amount`, `tip_amount`, `total_amount`, `congestion_surcharge`.


In [ ]:
import os 
import sqlite3
import pandas as pd
import numpy as np

BASE_DIR = os.path.dirname(os.getcwd()) 
db_path = os.path.join(BASE_DIR, "data", "raw", "2019-12.sqlite")
lookup_path = os.path.join("data", "taxi_zone_lookup.csv")
processed_csv = r"data/processed/2019-12.csv"
processed_sqlite = r"data/processed/2019-12.sqlite"

SAMPLE_LIMIT = None

In [ ]:
# Cargar datos desde SQLite (tabla 'tripdata')
# Usar el path absoluto definido en la celda 1
conn = sqlite3.connect(db_path)

if SAMPLE_LIMIT:
    query = f"SELECT * FROM tripdata LIMIT {SAMPLE_LIMIT}"
else:
    query = "SELECT * FROM tripdata;"

df = pd.read_sql_query(query, conn)
conn.close()

print("Registros cargados:", len(df))
print("Columnas cargadas:", df.columns.tolist())



In [ ]:
# Seleccionamos las columnas que queremos conservar

cols_keep = [
    'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'passenger_count', 'trip_distance',
    'pulocationid', 'dolocationid', 'ratecodeid',
    'payment_type', 'fare_amount', 'tip_amount',
    'total_amount', 'congestion_surcharge'
]

# Verificamos que todas existan 
cols_miss = [c for c in cols_keep if c not in df.columns]
if cols_miss:
    raise ValueError("Faltan columnas esperadas en la tabla;", cols_miss)

df = df[cols_keep].copy()
print("Shape tras seleccionar columnas:", df.shape)
df.head(3)


In [ ]:
# Convertir tipos: fechas a datetime, IDs a int, numeric clean 
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

#pulocationid y dolocationid a int para poder tener Na si hay 
df['pulocationid'] = df['pulocationid'].astype('Int64', copy=False)
df['dolocationid'] = df['dolocationid'].astype('Int64', copy=False)

# passenger_count a int 
df['passenger_count'] = df['passenger_count'].astype('Int64', copy=False)

# Asegurar columnas numericas donde corresponden 
numeric_cols = ['trip_distance', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount', 'congestion_surcharge']
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Revision rapida 
print(df.dtypes)
print("\nNulos por columna (después de parse):")
print(df.isna().sum())

#pulocationid y dolocationid a int para poder tener Na si hay 
df['pulocationid'] = df['pulocationid'].astype('Int64', copy=False)
df['dolocationid'] = df['dolocationid'].astype('Int64', copy=False)

# passenger_count a int 
df['passenger_count'] = df['passenger_count'].astype('Int64', copy=False)

# Asegurar columnas numericas donde corresponden 
numeric_cols = ['trip_distance', 'payment_type', 'fare_amount', 'tip_amount', 'total_amount', 'congestion_surcharge']
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Revision rapida 
print(df.dtypes)
print("\nNulos por columna (después de parse):")
print(df.isna().sum())

In [ ]:
# Eliminacion de nulos y registros invalidos 
df.dropna(subset=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'total_amount'], inplace=True)

# filtrar viajes con distacia, tarifa o monto negativos o null 
df = df[
    (df['trip_distance'] > 0) &
    (df['fare_amount'] > 0) &
    (df['total_amount'] > 0)
]
print("✅ Filas después de limpieza:", df.shape[0])
df.head()

In [ ]:
# Crear columna de duración del viaje en minutos
df["trip_duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

# Eliminamos viajes con duración negativa o cero
df = df[df["trip_duration_min"] > 0]

df["trip_duration_min"] = df["trip_duration_min"].round(2)

print("Duracion de viaje calculada y registros negativos eliminados.")
df[["trip_distance", "trip_duration_min"]].describe()


In [ ]:
# 1. ¿Cuántos viajes tienen passenger_count = 0?
pc_zero = df[df['passenger_count'] == 0].shape[0]
print(f"Viajes con passenger_count = 0: {pc_zero}")

# 2. Distribución de payment_type (incluyendo nulos)
print("\nDistribución de payment_type:")
print(df['payment_type'].value_counts(dropna=False))

# 3. Percentiles altos de trip_distance
print("\nPercentiles de trip_distance:")
print(df['trip_distance'].quantile([0.95, 0.99, 0.999, 0.9999]))

# 4. Percentiles altos de trip_duration_min
print("\nPercentiles de trip_duration_min:")
print(df['trip_duration_min'].quantile([0.95, 0.99, 0.999, 0.9999]))

# 5. Mostrar algunas filas con trip_distance extremadamente alto
print("\nViajes con trip_distance > 200:")
print(df[df['trip_distance'] > 200][['trip_distance', 'fare_amount', 'trip_duration_min']].head())

# 6. Mostrar algunas filas con trip_duration_min extremadamente alto
print("\nViajes con trip_duration_min > 500:")
print(df[df['trip_duration_min'] > 500][['trip_duration_min', 'trip_distance', 'fare_amount']].head())

In [ ]:
# 1️⃣ Eliminar passenger_count = 0
df = df[df['passenger_count'] != 0]

# 2️⃣ Eliminar payment_type NaN y payment_type = 5 (unknown)
df = df[~df['payment_type'].isna()]  # elimina NaN
df = df[df['payment_type'] != 5]     # elimina 5 (solo 1 fila)

# 3️⃣ Eliminar distancias imposibles (> 200 millas)
df = df[df['trip_distance'] <= 200]

# 4️⃣ Eliminar duraciones absurdas (> 500 minutos ≈ 8h 20min)
df = df[df['trip_duration_min'] <= 500]

# ✅ Reiniciar índice después de filtrado
df = df.reset_index(drop=True)

# ✅ Mostrar cuántos registros quedaron
print("Filas finales:", len(df))

In [ ]:
import os
BASE_DIR = os.path.dirname(os.getcwd()) 
processed_dir = os.path.join( BASE_DIR,"data", "processed")

# Asegurar que la carpeta existe (por si acaso)
os.makedirs(processed_dir, exist_ok=True)

# Ruta final del archivo CSV
output_path = os.path.join(processed_dir, "tripdata_clean.csv")

# Guardar el dataframe limpio
df.to_csv(output_path, index=False)

print(f" Datos limpios guardados correctamente en:\n{output_path}")
